# <h1 align="center"><font color="red">PDF and Images Content Extraction for Large Language Models</font></h1>

<font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este Notebook está baseado no artigo de [keitazoumana]().

# <font color="pink">pypdfium2</font>

In [1]:
# pip install pypdfium2 matplotlib pillow

import pypdfium2 as pdfium
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

def convert_pdf_to_images(file_path, scale=300/72):
    
    pdf_file = pdfium.PdfDocument(file_path)  
    page_indices = [i for i in range(len(pdf_file))]
    
    renderer = pdf_file.render(
        pdfium.PdfBitmap.to_pil,
        page_indices = page_indices, 
        scale = scale,
    )
    
    list_final_images = [] 
    
    for i, image in zip(page_indices, renderer):
        
        image_byte_array = BytesIO()
        image.save(image_byte_array, format='jpeg', optimize=True)
        image_byte_array = image_byte_array.getvalue()
        list_final_images.append(dict({i:image_byte_array}))
    
    return list_final_images

In [2]:
def display_images(list_dict_final_images):
    
    all_images = [list(data.values())[0] for data in list_dict_final_images]

    for index, image_bytes in enumerate(all_images):

        image = Image.open(BytesIO(image_bytes))
        figure = plt.figure(figsize = (image.width / 100, image.height / 100))

        plt.title(f"----- Page Number {index+1} -----")
        plt.imshow(image)
        plt.axis("off")
        plt.show()
        

In [3]:

convert_pdf_to_images = convert_pdf_to_images('./4._Ouvidoria_Geral_1.pdf')

#convert_pdf_to_images

In [ ]:
# Logo convertemos a Imagem o PDF:

# display_images(convert_pdf_to_images)

# <font color="pink">pytesseract</font>

In [ ]:
#!pip install pytesseract

from pytesseract import image_to_string 

def extract_text_with_pytesseract(list_dict_final_images):
    
    image_list = [list(data.values())[0] for data in list_dict_final_images]
    image_content = []
    
    for index, image_bytes in enumerate(image_list):
        
        image = Image.open(BytesIO(image_bytes))
        raw_text = str(image_to_string(image))
        image_content.append(raw_text)
    
    return "\n".join(image_content)

In [ ]:
text_with_pytesseract = extract_text_with_pytesseract(convert_pdf_to_images)

print(text_with_pytesseract)

# <font color="pink">EasyOCR</font>

In [4]:
#!pip install easyocr

from easyocr import Reader

# Load model for the English language
language_reader = Reader(["pt"])

In [5]:
def extract_text_with_easyocr(list_dict_final_images):
    
    image_list = [list(data.values())[0] for data in list_dict_final_images]
    image_content = []
    
    for index, image_bytes in enumerate(image_list):
        
        image = Image.open(BytesIO(image_bytes))
        raw_text = language_reader.readtext(image)
        raw_text = "\n".join([res[1] for res in raw_text])
                       
        image_content.append(raw_text)
    
    return "\n".join(image_content)

In [ ]:
text_with_easy_ocr = extract_text_with_easyocr(convert_pdf_to_images)

print(text_with_easy_ocr)

# <font color="pink">PyPDF</font>

In [19]:
#!pip install PyPDF2
from PyPDF2 import PdfReader

def extract_text_with_pyPDF(PDF_File):

    pdf_reader = PdfReader(PDF_File)
    
    raw_text = ''

    for i, page in enumerate(pdf_reader.pages):
        
        text = page.extract_text()
        if text:
            raw_text += text

    return raw_text

In [20]:
text_with_pyPDF = extract_text_with_pyPDF("./4._Ouvidoria_Geral_1.pdf")

print(text_with_pyPDF)

 
Controladoria -Geral do Distrito 
Federal  O QUE A CONTROLADORIA -
GERAL DO DF FAZ POR VOCÊ  
A Carta de Serviços ao Cidadão é um instrumento de controle social que facilita a sua 
participação nas ações e programas do Governo do Distrito Federal. Na Carta da Contro-
ladoria -Geral do DF você encontrará informações claras e acessíveis sobre quem somos, 
o que fazemos e os serviços que oferecemos. Bem informado, você poderá melhor exer-
cer sua cidadania ao acompanhar a execução das ações governamentais e avaliar os 
nossos compromissos assumidos.  
O foco é a transparência das informações do governo e a participação social. Por isto, 
estamos sempre buscando formas para você colaborar com os serviços públicos presta-
dos por qualquer instituição pública do Distrito Federal.  
Aqui, você também encontrará informações e orientações sobre como se comunicar com 
a Controladoria -Geral do DF e utilizar os seus serviços.   
Nossa ouvidoria está pronta para receber suas demandas e opiniões 

# <font color="pink">LangChain</font>

In [17]:
# !pip install langchain

from langchain.document_loaders.image import UnstructuredImageLoader

from PIL import Image
from io import BytesIO
import tempfile
import os

# def extract_text_with_langchain_image(list_dict_final_images):
    
#     image_list = [list(data.values())[0] for data in list_dict_final_images]
#     image_content = []
    
#     for index, image_bytes in enumerate(image_list):
        
#         image = Image.open(BytesIO(image_bytes))
#         loader = UnstructuredImageLoader(image)
#         data = loader.load()
#         raw_text = data[index].page_content
                       
#         image_content.append(raw_text)
    
#     return "\n".join(image_content)


def extract_text_with_langchain_image(list_dict_final_images):
    image_list = [list(data.values())[0] for data in list_dict_final_images]
    image_content = []
    
    for index, image_bytes in enumerate(image_list):
        image = Image.open(BytesIO(image_bytes))
        
        # Salva a imagem em um arquivo temporário
        with tempfile.NamedTemporaryFile(delete=False, suffix='.jpeg') as temp_image_file:
            image.save(temp_image_file, format='JPEG')
            temp_image_path = temp_image_file.name
        
        try:
            loader = UnstructuredImageLoader(temp_image_path)
            data = loader.load()
            raw_text = data[0].page_content  # Index 0 para obter o conteúdo da primeira página
            image_content.append(raw_text)
        finally:
            os.remove(temp_image_path)  # Remove o arquivo temporário
    
    return "\n".join(image_content)
    

In [18]:
text_with_langchain_image = extract_text_with_langchain_image(convert_pdf_to_images)

print(text_with_langchain_image)

Controladoria-Geral do Distrito Federal



CARTA DE SERVICOS AO CIDADAO



YX




O QUE A CONTROLADORIA- GERAL DO DF FAZ POR VOCE

A Carta de Servicos ao Cidadao é um instrumento de controle social que facilita a sua participagao nas agdes e programas do Governo do Distrito Federal. Na Carta da Contro- ladoria-Geral do DF vocé encontrara informagées claras e acessiveis sobre quem somos, 0 que fazemos e os servicgos que oferecemos. Bem informado, vocé podera melhor exer- cer sua cidadania ao acompanhar a execucgao das agées governamentais e avaliar os nossos compromissos assumidos.

O foco é a transparéncia das informagées do governo e a participacao social. Por isto, estamos sempre buscando formas para vocé colaborar com os servigos pUblicos presta- dos por qualquer instituigao publica do Distrito Federal.

Aqui, vocé também encontrara informagées e orientagdes sobre como se comunicar com a Controladoria-Geral do DF e utilizar os seus servigos.

Nossa ouvidoria esta pronta para receber

In [6]:
from langchain.document_loaders import UnstructuredFileLoader

def extract_text_with_langchain_pdf(pdf_file):
    
    loader = UnstructuredFileLoader(pdf_file)
    documents = loader.load()
    pdf_pages_content = '\n'.join(doc.page_content for doc in documents)
    
    return pdf_pages_content
    

In [7]:
text_with_langchain_files = extract_text_with_langchain_pdf("./4._Ouvidoria_Geral_1.pdf")

print(text_with_langchain_files)

Controladoria-Geral do Distrito Federal

O QUE A CONTROLADORIA- GERAL DO DF FAZ POR VOCÊ

A Carta de Serviços ao Cidadão é um instrumento de controle social que facilita a sua

participação nas ações e programas do Governo do Distrito Federal. Na Carta da Contro-

ladoria-Geral do DF você encontrará informações claras e acessíveis sobre quem somos,

o que fazemos e os serviços que oferecemos. Bem informado, você poderá melhor exer-

cer sua cidadania ao acompanhar a execução das ações governamentais e avaliar os

nossos compromissos assumidos.

O foco é a transparência das informações do governo e a participação social. Por isto,

estamos sempre buscando formas para você colaborar com os serviços públicos presta-

dos por qualquer instituição pública do Distrito Federal.

Aqui, você também encontrará informações e orientações sobre como se comunicar com

a Controladoria-Geral do DF e utilizar os seus serviços.

Nossa ouvidoria está pronta para receber suas demandas e opiniões também so